# Pre-built cadCAD Models

This notebook shows how a data scientist can take cadCAD models "off the rack" from an MSML dev without ever needing to get into the details.

In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

from cadCADModels import cadCAD_models

## Models

We have the different models that the MSML dev has created which we can import simply by pulling in cadCAD_models. Each comes with its state space, parameter space, and the model that actually runs the simulation.

We have two options here, Model 1 and Model 2 which can be toggled between. The only difference is model 2 only has two of the blocks used.

There is a third model covered afterwards that shows how certain state and parameters can be pre-assigned.

In [3]:
state_space["Dummy"].__annotations__

{'Words': str, 'Total Length': int}

In [4]:
print(state_space["Dummy"])

<class 'src.TypeMappings.types.EntityType'>


In [2]:
# Get model components
model_name = "Model 1"
state_space = cadCAD_models[model_name]["StateSpace"]
parameter_space = cadCAD_models[model_name]["ParameterSpace"]
model = cadCAD_models[model_name]["Model"]

# Printing model data
print("Model blocks: {}".format(model.blocks))
print()
print("Model state space:")
display(state_space)
print()
print("Model parameter space:")
display(parameter_space)

Model blocks: ['DUMMY Length-2 Boundary Wiring', 'DUMMY Length-1 Boundary Wiring', 'DUMMY Control Wiring']

Model state space:


{'Dummy': src.TypeMappings.types.EntityType,
 'Time': int,
 'Simulation Log': typing.List[src.TypeMappings.types.SimulationLogEntry]}


Model parameter space:


{'DUMMY D Probability': float,
 'DUMMY Length Multiplier': int,
 'FP DUMMY Length-2 ABC Combo Boundary Action': typing.Literal['DUMMY Length-2 ABC Equal Weight Option', 'DUMMY Length-2 ABC Equal Weight 2 Option'],
 'FP DUMMY Length-1 DEF Control Action': typing.Literal['DUMMY Length-1 DEF Equal Weight Option', 'DUMMY Length-1 DEF D Probability Option']}

### Single Experiment

A single experiment is as easily run as below. We use deepcopy and recording of trajectories but we can also switch them off if we want to just simply take the ending state and use something like a simulation log to record the data.

Please note that this version does have a simulation log, but it can be ignored as we are doing a logging at each step with deepcopy of the entire state.

In [3]:
# Create expriment and run the model
state: state_space = {'Dummy': {'Words': '', 'Total Length': None},
                                   'Time': 0,
                                   'Simulation Log': []}
params: parameter_space = {'DUMMY D Probability': 0.5,
 'DUMMY Length Multiplier': 3,
 'FP DUMMY Length-1 DEF Control Action': 'DUMMY Length-1 DEF Equal Weight Option',
 'FP DUMMY Length-2 ABC Combo Boundary Action': 'DUMMY Length-2 ABC Equal Weight Option'}

experiment = model.create_experiment(state, params, record_trajectory=True, use_deepcopy=True)
experiment.run(5)
trajectories = experiment.trajectories
print("The last two trajectories:")
display(experiment.trajectories[-2:])
print()
print()
print("Current state:")
display(experiment.state)

The last two trajectories:


[{'Dummy': {'Words': 'CAADBCAFCABDCAAF', 'Total Length': 48},
  'Time': 12,
  'Simulation Log': [{'Time': 1, 'Word': 'CA', 'Length (Multiplied)': 6},
   {'Time': 2, 'Word': 'CAA', 'Length (Multiplied)': 9},
   {'Time': 3, 'Word': 'CAAD', 'Length (Multiplied)': 12},
   {'Time': 4, 'Word': 'CAADBC', 'Length (Multiplied)': 18},
   {'Time': 5, 'Word': 'CAADBCA', 'Length (Multiplied)': 21},
   {'Time': 6, 'Word': 'CAADBCAF', 'Length (Multiplied)': 24},
   {'Time': 7, 'Word': 'CAADBCAFCA', 'Length (Multiplied)': 30},
   {'Time': 8, 'Word': 'CAADBCAFCAB', 'Length (Multiplied)': 33},
   {'Time': 9, 'Word': 'CAADBCAFCABD', 'Length (Multiplied)': 36},
   {'Time': 10, 'Word': 'CAADBCAFCABDCA', 'Length (Multiplied)': 42},
   {'Time': 11, 'Word': 'CAADBCAFCABDCAA', 'Length (Multiplied)': 45},
   {'Time': 12, 'Word': 'CAADBCAFCABDCAAF', 'Length (Multiplied)': 48}]},
 {'Dummy': {'Words': 'CAADBCAFCABDCAAFBCBD', 'Total Length': 60},
  'Time': 15,
  'Simulation Log': [{'Time': 1, 'Word': 'CA', 'Length 



Current state:


{'Dummy': {'Words': 'CAADBCAFCABDCAAFBCBD', 'Total Length': 60},
 'Time': 15,
 'Simulation Log': [{'Time': 1, 'Word': 'CA', 'Length (Multiplied)': 6},
  {'Time': 2, 'Word': 'CAA', 'Length (Multiplied)': 9},
  {'Time': 3, 'Word': 'CAAD', 'Length (Multiplied)': 12},
  {'Time': 4, 'Word': 'CAADBC', 'Length (Multiplied)': 18},
  {'Time': 5, 'Word': 'CAADBCA', 'Length (Multiplied)': 21},
  {'Time': 6, 'Word': 'CAADBCAF', 'Length (Multiplied)': 24},
  {'Time': 7, 'Word': 'CAADBCAFCA', 'Length (Multiplied)': 30},
  {'Time': 8, 'Word': 'CAADBCAFCAB', 'Length (Multiplied)': 33},
  {'Time': 9, 'Word': 'CAADBCAFCABD', 'Length (Multiplied)': 36},
  {'Time': 10, 'Word': 'CAADBCAFCABDCA', 'Length (Multiplied)': 42},
  {'Time': 11, 'Word': 'CAADBCAFCABDCAA', 'Length (Multiplied)': 45},
  {'Time': 12, 'Word': 'CAADBCAFCABDCAAF', 'Length (Multiplied)': 48},
  {'Time': 13, 'Word': 'CAADBCAFCABDCAAFBC', 'Length (Multiplied)': 54},
  {'Time': 14, 'Word': 'CAADBCAFCABDCAAFBCB', 'Length (Multiplied)': 57},


## Batch Experiments

Batch experiments can be run like below:

In [4]:
from typing import List
state: list = [{'Dummy': {'Words': '', 'Total Length': None},
                                   'Time': 0,
                                   'Simulation Log': []},
                            {'Dummy': {'Words': '', 'Total Length': None},
                                   'Time': 0,
                                   'Simulation Log': []}]
params: list = [{'DUMMY D Probability': 0.5,
 'DUMMY Length Multiplier': 3,
 'FP DUMMY Length-1 DEF Control Action': 'DUMMY Length-1 DEF Equal Weight Option',
 'FP DUMMY Length-2 ABC Combo Boundary Action': 'DUMMY Length-2 ABC Equal Weight Option'},
 {'DUMMY D Probability': 0.1,
 'DUMMY Length Multiplier': 3,
 'FP DUMMY Length-1 DEF Control Action': 'DUMMY Length-1 DEF Equal Weight Option',
 'FP DUMMY Length-2 ABC Combo Boundary Action': 'DUMMY Length-2 ABC Equal Weight Option'}]
batch_experiments = model.create_batch_experiments(state, params, record_trajectory=True, use_deepcopy=True)
batch_experiments.run(3)
print("Trajectories (2 experiments x 4 entries including starting trajectory)")
display(batch_experiments.trajectories)
print()
print("Confirmation that the shape of trajectories is: {} x {}".format(len(batch_experiments.trajectories), len(batch_experiments.trajectories[0])))


Trajectories (2 experiments x 4 entries including starting trajectory)


[[{'Dummy': {'Words': '', 'Total Length': 0}, 'Time': 0, 'Simulation Log': []},
  {'Dummy': {'Words': 'BABD', 'Total Length': 12},
   'Time': 3,
   'Simulation Log': [{'Time': 1, 'Word': 'BA', 'Length (Multiplied)': 6},
    {'Time': 2, 'Word': 'BAB', 'Length (Multiplied)': 9},
    {'Time': 3, 'Word': 'BABD', 'Length (Multiplied)': 12}]},
  {'Dummy': {'Words': 'BABDCCCF', 'Total Length': 24},
   'Time': 6,
   'Simulation Log': [{'Time': 1, 'Word': 'BA', 'Length (Multiplied)': 6},
    {'Time': 2, 'Word': 'BAB', 'Length (Multiplied)': 9},
    {'Time': 3, 'Word': 'BABD', 'Length (Multiplied)': 12},
    {'Time': 4, 'Word': 'BABDCC', 'Length (Multiplied)': 18},
    {'Time': 5, 'Word': 'BABDCCC', 'Length (Multiplied)': 21},
    {'Time': 6, 'Word': 'BABDCCCF', 'Length (Multiplied)': 24}]},
  {'Dummy': {'Words': 'BABDCCCFCBBE', 'Total Length': 36},
   'Time': 9,
   'Simulation Log': [{'Time': 1, 'Word': 'BA', 'Length (Multiplied)': 6},
    {'Time': 2, 'Word': 'BAB', 'Length (Multiplied)': 9},
 


Confirmation that the shape of trajectories is: 2 x 4


## Model with Pre-assigned State & Parameters

In [5]:
# Get model components
model_name = "Model 3"
state_space = cadCAD_models[model_name]["StateSpace"]
parameter_space = cadCAD_models[model_name]["ParameterSpace"]
model = cadCAD_models[model_name]["Model"]

# Printing model data
print("Model blocks: {}".format(model.blocks))
print()
print("Model state space:")
display(state_space)
print()
print("Model parameter space:")
display(parameter_space)

Model blocks: ['DUMMY Length-2 Boundary Wiring', 'DUMMY Length-1 Boundary Wiring', 'DUMMY Control Wiring']

Model state space:


{'Dummy': src.TypeMappings.types.EntityType,
 'Simulation Log': typing.List[src.TypeMappings.types.SimulationLogEntry]}


Model parameter space:


{'DUMMY D Probability': float, 'DUMMY Length Multiplier': int}

In [6]:
# Create expriment and run the model
state: state_space = {'Dummy': {'Words': '', 'Total Length': None},
                                   'Simulation Log': []}
params: parameter_space = {'DUMMY D Probability': 0.5,
 'DUMMY Length Multiplier': 3,}

experiment = model.create_experiment(state, params, record_trajectory=True, use_deepcopy=True)
experiment.run(5)
trajectories = experiment.trajectories
print("The last two trajectories:")
display(experiment.trajectories[-2:])
print()
print()
print("Current state:")
display(experiment.state)

The last two trajectories:


[{'Dummy': {'Words': 'BCBFBBCFCBBDAACE', 'Total Length': 48},
  'Simulation Log': [{'Time': 1, 'Word': 'BC', 'Length (Multiplied)': 6},
   {'Time': 2, 'Word': 'BCB', 'Length (Multiplied)': 9},
   {'Time': 3, 'Word': 'BCBF', 'Length (Multiplied)': 12},
   {'Time': 4, 'Word': 'BCBFBB', 'Length (Multiplied)': 18},
   {'Time': 5, 'Word': 'BCBFBBC', 'Length (Multiplied)': 21},
   {'Time': 6, 'Word': 'BCBFBBCF', 'Length (Multiplied)': 24},
   {'Time': 7, 'Word': 'BCBFBBCFCB', 'Length (Multiplied)': 30},
   {'Time': 8, 'Word': 'BCBFBBCFCBB', 'Length (Multiplied)': 33},
   {'Time': 9, 'Word': 'BCBFBBCFCBBD', 'Length (Multiplied)': 36},
   {'Time': 10, 'Word': 'BCBFBBCFCBBDAA', 'Length (Multiplied)': 42},
   {'Time': 11, 'Word': 'BCBFBBCFCBBDAAC', 'Length (Multiplied)': 45},
   {'Time': 12, 'Word': 'BCBFBBCFCBBDAACE', 'Length (Multiplied)': 48}],
  'Time': 12},
 {'Dummy': {'Words': 'BCBFBBCFCBBDAACEBBAD', 'Total Length': 60},
  'Simulation Log': [{'Time': 1, 'Word': 'BC', 'Length (Multiplied)':



Current state:


{'Dummy': {'Words': 'BCBFBBCFCBBDAACEBBAD', 'Total Length': 60},
 'Simulation Log': [{'Time': 1, 'Word': 'BC', 'Length (Multiplied)': 6},
  {'Time': 2, 'Word': 'BCB', 'Length (Multiplied)': 9},
  {'Time': 3, 'Word': 'BCBF', 'Length (Multiplied)': 12},
  {'Time': 4, 'Word': 'BCBFBB', 'Length (Multiplied)': 18},
  {'Time': 5, 'Word': 'BCBFBBC', 'Length (Multiplied)': 21},
  {'Time': 6, 'Word': 'BCBFBBCF', 'Length (Multiplied)': 24},
  {'Time': 7, 'Word': 'BCBFBBCFCB', 'Length (Multiplied)': 30},
  {'Time': 8, 'Word': 'BCBFBBCFCBB', 'Length (Multiplied)': 33},
  {'Time': 9, 'Word': 'BCBFBBCFCBBD', 'Length (Multiplied)': 36},
  {'Time': 10, 'Word': 'BCBFBBCFCBBDAA', 'Length (Multiplied)': 42},
  {'Time': 11, 'Word': 'BCBFBBCFCBBDAAC', 'Length (Multiplied)': 45},
  {'Time': 12, 'Word': 'BCBFBBCFCBBDAACE', 'Length (Multiplied)': 48},
  {'Time': 13, 'Word': 'BCBFBBCFCBBDAACEBB', 'Length (Multiplied)': 54},
  {'Time': 14, 'Word': 'BCBFBBCFCBBDAACEBBA', 'Length (Multiplied)': 57},
  {'Time': 15